<a href="https://colab.research.google.com/github/LokeshVadlamudi/DeepLearningClass/blob/master/dl1mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np

In [2]:
from keras.datasets import mnist

Using TensorFlow backend.


In [3]:
(x_train,y_train) , (x_test,y_test) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [0]:
images, labels = (x_train[0:1000].reshape(1000,28*28))/255,y_train[0:1000]

In [0]:
one_hot_labels = np.zeros((len(labels),10))

In [0]:
for i,l in enumerate(labels):
  one_hot_labels[i][l] = 1
labels = one_hot_labels

In [0]:
test_images = x_test.reshape(len(x_test),28*28)/255
test_labels = np.zeros((len(y_test),10))


In [0]:
for i,l in enumerate(y_test):
  test_labels[i][l] = 1

In [0]:
def tanh(x):
  return np.tanh(x)

In [0]:
def tanh2deriv(output):
  return 1 - (output ** 2)

In [0]:
def softmax(x):
  temp = np.exp(x)
  return temp / np.sum(temp,axis = 1, keepdims = True)

In [0]:
alpha , iterations , hidden_size = (2,300,100)

In [0]:
ppi , num_labels = (784,10)

In [0]:
batch_size = 100

In [0]:
weights_0_1 = 0.02*np.random.random((ppi,hidden_size))-0.01

In [0]:
weights_1_2 = 0.2*np.random.random((hidden_size,num_labels))-0.1

In [20]:
for j in range(iterations):
  correct_cnt = 0
  for i in range(int(len(images) / batch_size)):
    batch_start, batch_end = ((i * batch_size), ((i + 1)*batch_size))
    layer_0 = images[batch_start:batch_end]
    layer_1 = tanh(np.dot(layer_0,weights_0_1))
    dropout_mask = np.random.randint(2,size=layer_1.shape)
    layer_1 *= dropout_mask*2
    layer_2 = softmax(np.dot(layer_1,weights_1_2))

    for k in range(batch_size):
      correct_cnt += int(np.argmax(layer_2[k:k+1])== np.argmax(labels[batch_start+k:batch_start+k+1]))
    
    layer_2_delta = (labels[batch_start:batch_end] - layer_2) / (batch_size * layer_2.shape[0])
    layer_1_delta = layer_2_delta.dot(weights_1_2.T) * tanh2deriv(layer_1)

    layer_1_delta *= dropout_mask

    weights_1_2 += alpha * layer_1.T.dot(layer_2_delta)
    weights_0_1 += alpha * layer_0.T.dot(layer_1_delta)

  test_correct_cnt = 0
  
  for i in range(len(test_images)):
    
    layer_0 = test_images[i:i+1]
    layer_1 = tanh(np.dot(layer_0,weights_0_1))
    layer_2 = np.dot(layer_1,weights_1_2)
    test_correct_cnt += int(np.argmax(layer_2) == np.argmax(test_labels[i:i+1]))
  
  if(j % 10 == 0):
    print(str(j) + " test_acc" + str(test_correct_cnt/float(len(test_images))) + " Train_acc" + str(correct_cnt/float(len(images))))







0 test_acc0.8699 Train_acc0.944
10 test_acc0.871 Train_acc0.948
20 test_acc0.8721 Train_acc0.952
30 test_acc0.8727 Train_acc0.951
40 test_acc0.8732 Train_acc0.955
50 test_acc0.8736 Train_acc0.958
60 test_acc0.8747 Train_acc0.949
70 test_acc0.8756 Train_acc0.956
80 test_acc0.8774 Train_acc0.937
90 test_acc0.8758 Train_acc0.941
100 test_acc0.8758 Train_acc0.954
110 test_acc0.8761 Train_acc0.957
120 test_acc0.8775 Train_acc0.956
130 test_acc0.8781 Train_acc0.949
140 test_acc0.8768 Train_acc0.954
150 test_acc0.8787 Train_acc0.952
160 test_acc0.8782 Train_acc0.954
170 test_acc0.878 Train_acc0.957
180 test_acc0.8791 Train_acc0.955
190 test_acc0.8794 Train_acc0.955
200 test_acc0.8802 Train_acc0.951
210 test_acc0.8784 Train_acc0.957
220 test_acc0.8808 Train_acc0.963
230 test_acc0.8815 Train_acc0.966
240 test_acc0.8815 Train_acc0.964
250 test_acc0.8817 Train_acc0.961
260 test_acc0.8822 Train_acc0.965
270 test_acc0.8821 Train_acc0.954
280 test_acc0.8819 Train_acc0.954
290 test_acc0.8821 Train_ac